In [122]:
import pymcdm
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

In [123]:
constructors = json.load(open('./data/constructors.json'))
drivers = json.load(open('./data/drivers.json'))

DRIVERS

In [124]:
names = [d['driver'] for d in drivers]
startPrice = [d['startPrice'] for d in drivers]
race_points = []
race_price = []
for race in range(len(drivers[0]['races'])):
    race_points.append([d['races'][race]['points']['total'] for d in drivers])
    race_price.append([d['races'][race]['price'] for d in drivers])

In [125]:
df = pd.DataFrame([names, startPrice, *race_points, *race_price], ['name', 'startPrice', 'points', 'newPrice']).T
df

,name,startPrice,points,newPrice
0,Max Verstappen,26.9,35,27
1,Fernando Alonso,8.3,39,8.5
2,Charles Leclerc,21.2,-11,21.1
3,Zhou Guanyu,4.9,15,4.9
4,Nico Hulkenberg,4.3,-1,4.2
5,Sergio Perez,18,28,18.1
6,Esteban Ocon,9.4,-17,9.3
7,Pierre Gasly,8.2,20,8.2
8,Carlos Sainz,17.2,19,17.2
9,Lewis Hamilton,23.7,19,23.7


In [126]:
matrix = df.to_numpy()
matrix = matrix[:, 1:].astype(float)

In [127]:
# weights = pymcdm.weights.standard_deviation_weights(matrix)
weights = np.array([0.2, 0.65, 0.15])
types = np.array([-1, 1, -1])

TOPSIS = pymcdm.methods.TOPSIS()

pref = TOPSIS(matrix, weights, types)
rank = pymcdm.helpers.rrankdata(pref)

In [128]:
df['ranking'] = rank
df

,name,startPrice,points,newPrice,ranking
0,Max Verstappen,26.9,35,27,3.0
1,Fernando Alonso,8.3,39,8.5,1.0
2,Charles Leclerc,21.2,-11,21.1,20.0
3,Zhou Guanyu,4.9,15,4.9,6.0
4,Nico Hulkenberg,4.3,-1,4.2,16.0
5,Sergio Perez,18,28,18.1,2.0
6,Esteban Ocon,9.4,-17,9.3,19.0
7,Pierre Gasly,8.2,20,8.2,4.0
8,Carlos Sainz,17.2,19,17.2,7.0
9,Lewis Hamilton,23.7,19,23.7,10.0


In [129]:
BUDGET = 100
DRIVERS_AMOUNT = 5

df = df.sort_values(by=['ranking'])

drivers_set = pd.DataFrame(columns=df.columns)

for index, driver in df.iterrows():
    if BUDGET - driver['startPrice'] > 0:
        entry = df.loc[df['name'] == driver['name']]
        drivers_set = pd.concat([drivers_set, entry])
        BUDGET -= driver['startPrice']

    if drivers_set.shape[0] == DRIVERS_AMOUNT:
        break


In [130]:
drivers_set

,name,startPrice,points,newPrice,ranking
1,Fernando Alonso,8.3,39,8.5,1.0
5,Sergio Perez,18,28,18.1,2.0
0,Max Verstappen,26.9,35,27,3.0
7,Pierre Gasly,8.2,20,8.2,4.0
10,Lance Stroll,7.5,17,7.6,5.0


In [131]:
BUDGET

31.10000000000001

CONSTRUCTORS

In [132]:
team = [c['team'] for c in constructors]
startPrice = [c['startPrice'] for c in constructors]
race_points = []
race_price = []
for race in range(len(constructors[0]['races'])):
    race_points.append([c['races'][race]['points']['total'] for c in constructors])
    race_price.append([c['races'][race]['price'] for c in constructors])

In [133]:
constructors_df = pd.DataFrame([team, startPrice, *race_points, *race_price], ['team', 'startPrice', 'points', 'newPrice']).T
constructors_df

,team,startPrice,points,newPrice
0,Red Bull,27.2,78,27.3
1,Aston Martin,6.7,52,6.8
2,Ferrari,22.1,31,22.1
3,Alpine,10.1,-3,10
4,Mercedes,25.1,44,25.1
5,Alfa Romeo,6.2,30,6.2
6,Williams,5.1,16,5.1
7,Haas,5.3,13,5.3
8,McLaren,9.1,-10,9


In [134]:
matrix = constructors_df.to_numpy()
matrix = matrix[:, 1:].astype(float)

In [135]:
# weights = pymcdm.weights.standard_deviation_weights(matrix)
weights = np.array([0.2, 0.65, 0.15])
types = np.array([-1, 1, -1])

TOPSIS = pymcdm.methods.TOPSIS()

pref = TOPSIS(matrix, weights, types)
rank = pymcdm.helpers.rrankdata(pref)

In [136]:
constructors_df['ranking'] = rank
constructors_df

,team,startPrice,points,newPrice,ranking
0,Red Bull,27.2,78,27.3,2.0
1,Aston Martin,6.7,52,6.8,1.0
2,Ferrari,22.1,31,22.1,5.0
3,Alpine,10.1,-3,10,8.0
4,Mercedes,25.1,44,25.1,3.0
5,Alfa Romeo,6.2,30,6.2,4.0
6,Williams,5.1,16,5.1,6.0
7,Haas,5.3,13,5.3,7.0
8,McLaren,9.1,-10,9,9.0


In [137]:
BUDGET = 100
CONSTRUCTORS_AMOUNT = 2

constructors_df = constructors_df.sort_values(by=['ranking'])

constructors_set = pd.DataFrame(columns=constructors_df.columns)

for index, driver in constructors_df.iterrows():
    if BUDGET - driver['startPrice'] > 0:
        entry = constructors_df.loc[constructors_df['team'] == driver['team']]
        constructors_set = pd.concat([constructors_set, entry])
        BUDGET -= driver['startPrice']

    if constructors_set.shape[0] == CONSTRUCTORS_AMOUNT:
        break


In [138]:
constructors_set

,team,startPrice,points,newPrice,ranking
1,Aston Martin,6.7,52,6.8,1.0
0,Red Bull,27.2,78,27.3,2.0


In [139]:
BUDGET

66.1

TEAM SELECTION

FIRST VARIANT - primary selection for team

In [140]:
BUDGET = 100
DRIVERS_AMOUNT = 5
CONSTRUCTORS_AMOUNT = 2

constructors_df = constructors_df.sort_values(by=['ranking'])
constructors_set = pd.DataFrame(columns=constructors_df.columns)

df = df.sort_values(by=['ranking'])
drivers_set = pd.DataFrame(columns=df.columns)

for index, driver in constructors_df.iterrows():
    if BUDGET - driver['startPrice'] > 0:
        entry = constructors_df.loc[constructors_df['team'] == driver['team']]
        constructors_set = pd.concat([constructors_set, entry])
        BUDGET -= driver['startPrice']

    if constructors_set.shape[0] == CONSTRUCTORS_AMOUNT:
        break

for index, driver in df.iterrows():
    if BUDGET - driver['startPrice'] > 0:
        entry = df.loc[df['name'] == driver['name']]
        drivers_set = pd.concat([drivers_set, entry])
        BUDGET -= driver['startPrice']

    if drivers_set.shape[0] == DRIVERS_AMOUNT:
        break


In [141]:
constructors_set

,team,startPrice,points,newPrice,ranking
1,Aston Martin,6.7,52,6.8,1.0
0,Red Bull,27.2,78,27.3,2.0


In [142]:
drivers_set

,name,startPrice,points,newPrice,ranking
1,Fernando Alonso,8.3,39,8.5,1.0
5,Sergio Perez,18,28,18.1,2.0
0,Max Verstappen,26.9,35,27,3.0
7,Pierre Gasly,8.2,20,8.2,4.0
14,Logan Sargeant,4,11,4,12.0


In [143]:
points = constructors_set['points'].sum() + drivers_set['points'].sum()
points

263

In [144]:
BUDGET

0.6999999999999993

SECOND VARIANT - primary selection for drivers

In [145]:
BUDGET = 100
DRIVERS_AMOUNT = 5
CONSTRUCTORS_AMOUNT = 2

constructors_df = constructors_df.sort_values(by=['ranking'])
constructors_set = pd.DataFrame(columns=constructors_df.columns)

df = df.sort_values(by=['ranking'])
drivers_set = pd.DataFrame(columns=df.columns)

for index, driver in df.iterrows():
    if BUDGET - driver['startPrice'] > 0:
        entry = df.loc[df['name'] == driver['name']]
        drivers_set = pd.concat([drivers_set, entry])
        BUDGET -= driver['startPrice']

    if drivers_set.shape[0] == DRIVERS_AMOUNT:
        break
    
for index, driver in constructors_df.iterrows():
    if BUDGET - driver['startPrice'] > 0:
        entry = constructors_df.loc[constructors_df['team'] == driver['team']]
        constructors_set = pd.concat([constructors_set, entry])
        BUDGET -= driver['startPrice']

    if constructors_set.shape[0] == CONSTRUCTORS_AMOUNT:
        break



In [146]:
drivers_set

,name,startPrice,points,newPrice,ranking
1,Fernando Alonso,8.3,39,8.5,1.0
5,Sergio Perez,18,28,18.1,2.0
0,Max Verstappen,26.9,35,27,3.0
7,Pierre Gasly,8.2,20,8.2,4.0
10,Lance Stroll,7.5,17,7.6,5.0


In [147]:
constructors_set

,team,startPrice,points,newPrice,ranking
1,Aston Martin,6.7,52,6.8,1.0
5,Alfa Romeo,6.2,30,6.2,4.0


In [148]:
BUDGET

18.20000000000001

In [149]:
points = constructors_set['points'].sum() + drivers_set['points'].sum()
points

221